# This notebook serves as walkthrough for planning an experiment for creation through the OT2.
### The following modules are used and should be in the same directory as this notebook: 
* **CreateSamples** is responsible for calculating sample information, which includes component weight fractions and stock volumes
* **OT2Commands** is responsible for setting up information to be interpretted and executed by opentrons.
* **OT2Graphing** contains graphing tools to help visualize and explore parameter spaces.

In [251]:
import CreateSamples
import OT2Commands as ALH
import OT2Graphing as ographing
from opentrons import simulate, execute, protocol_api

# Would not load
import importlib # for reloading packages
import pandas as pd
import matplotlib.pyplot as plt

In [252]:
importlib.reload(CreateSamples)
importlib.reload(ALH)
importlib.reload(ographing)

<module 'OT2Graphing' from 'C:\\Users\\Edwin\\Desktop\\OT2-DOE\\Plan and Prepare\\OT2Graphing.py'>

## Step 1: Set up the experiment dictionary.
* The first step to planning an experiment is to load the experiment variables and inputs from a csv file. Every variable should have an input with an acceptable datatype. At the moment this step is done by opening a CSV file in Excel, where the first column is the name of the variable and the adjacent column is the variable value. The default delimitter is (,). 
    * Reading directly as csv is fine but it requires you have all data values in a string so then we can use ast.literal_eval to unpack this and the appropiate datatypes. This forces you \to put marks ('') around each variable value when planning the experiment. NOTE: You still need to place marks around anything inside a dtype i.e components inside a list.
        * To remove this dependency we can build our own interpreters for our specfic cases such as to not use ast.literal_evals default unpacking.
    * Loading from excel can be done in a similar manner but is avoided due not having xlrd or openpyxl depdenency native to python, and the opentrons being limited in the packages we can add/update. Hence we default to a CSV.
* **The experiment dictionary consist of keys being the variable name and the value being the variables value.**

In [253]:
path = r"C:\Users\Edwin\Desktop\OT2-DOE\Plan and Prepare\Testing Plans\02_25_21_Ouzo_Search.csv"
experiment_csv_dict = CreateSamples.get_experiment_plan(path) 

## Step 1a Optional: Load custom labware dictionary (Remote Testing)
* Provide the path to the directory holding all custom labware. This directory should have custom labware .json files you have previously made and tested, read more here: https://support.opentrons.com/en/articles/3136504-creating-custom-labware-definitions
* The reason we provide this is when working on a device that is not connected to the OT2's Jupyter notebook there is no way to natively use custom labware. So we create a dictionary of custom labware so we can later load into our protocol to primarily simualte and test protocols for execution later once connnected to the OT2's notebook.
    * When using custom labware on the OT2's notebook it pulls from a folder labeled "labware", which is something built into the Opentrons hardware. It has not been tested if the custom labware dictionary will superceed this directory of custom labware if used on the OT2.

In [254]:
labware_dir_path = r"C:\Users\Edwin\Desktop\OT2-DOE\Plan and Prepare\Custom Labware"
custom_labware_dict = ALH.custom_labware_dict(labware_dir_path)

## Step 2: Select and Create Sampling Space
* Create sampling space depending on the units of concentration and method of sampling. All information is pulled from the experimental dictionary made in Step 1.
    * Currently the only sampling method available are simple lattice and random based sampling. There are two potential ways to create samples in a system of n components which currently utilzie the linspaces of concentration. 
        * *Remember the linspace of concentration refers to [minimum concentration, maximum concentration, concentration step size]*
    * **Case 1 (Completing case):** Specify all but one (in this case the last) component's concentrations, which with the addition of exposing the unity_filter = True, would calculate the the remaining concentraiton values using the information of the last index of all component related variables (i.e. names). This is only meant for units that require unity like volf, wtf, and molf. 
    * **Case 2 (Non-completing case):** Specify all concentration linspaces, not applying any unit based filters, meant for all other non interdepedent units like molarity and mg/mL.
* Other things to take into consideration: All units must be the same for unity based, but not for non-unity units.

In [255]:
wtf_sample_canidates = CreateSamples.generate_candidate_lattice_concentrations(experiment_csv_dict, unity_filter=True)
# o
wtf_sample_canidates
# something interesting you can do here is after making these wtf samples you can now apply some random selection

,dppc wtf,dspepeg200 wtf,pfh wtf,ethanol wtf,water wtf
0,0.0001,0.000035,0.000,0.000000,0.999865
1,0.0001,0.000035,0.000,0.052632,0.947233
2,0.0001,0.000035,0.000,0.105263,0.894602
3,0.0001,0.000035,0.000,0.157895,0.841970
4,0.0001,0.000035,0.000,0.210526,0.789339
...,...,...,...,...,...
375,0.0001,0.000035,0.001,0.736842,0.262023
376,0.0001,0.000035,0.001,0.789474,0.209391
377,0.0001,0.000035,0.001,0.842105,0.156760
378,0.0001,0.000035,0.001,0.894737,0.104128


## Step 3: Calculate Volumes of Stocks
* From the concentration values calculated in Step 2, we use those along with stock concentration information to calculate the volume of required for each sample.

* This is where things get less "*general*" each case depending on the number of stocks, common components (i.e. component A in both stock A and B) and other requirements will typically require its own function. Luckily given the commmonality of using data frames this should be quite simple. 
* Currently the only function to calculate volumes in centered around the Ouzo emulsion systems. This system consist of 3 stock with the solvent being ethanol and two pure stocks of ethanol and water. 

* Ideally the way volumes should calculated is simply by calculating "*essential information*" given the concentration of component and the systems overall mass or volume. Using this essential information and the concentration unit of the stock, it should call the appropiate function to calcualte the volume. Many issue could arise such as having a molarity and providing a mass so would need to make sure these cases are sorted and reported back


In [256]:
volume_sample_canidates = CreateSamples.calculate_ouzo_volumes_from_wtf(wtf_sample_canidates, experiment_csv_dict)
volume_sample_canidates

,dppc-ethanol-stock uL,dspepeg2000-ethanol-stock uL,pfh-ethanol-stock uL,ethanol-stock uL,water-stock uL,Total Sample Volume uL
0,40.000002,33.267986,0.000000,-73.267988,799.892000,799.892000
1,40.000002,33.267986,0.000000,-31.612767,757.786737,799.441958
2,40.000002,33.267986,0.000000,9.469235,715.681474,798.418697
3,40.000002,33.267986,0.000000,49.914780,673.576211,796.758979
4,40.000002,33.267986,0.000000,89.655321,631.470947,794.394256
...,...,...,...,...,...,...
375,40.000002,33.267986,202.531646,231.028321,209.618316,716.446270
376,40.000002,33.267986,202.531646,258.807338,167.513053,702.120024
377,40.000002,33.267986,202.531646,285.312365,125.407789,686.519788
378,40.000002,33.267986,202.531646,310.572769,83.302526,669.674929


## Step 4: Create Complete Component Cocentration and Volume Dataframe and Apply Filters

In [257]:
complete_df = CreateSamples.combine_df(wtf_sample_canidates, volume_sample_canidates) # unfiltered
# complete_df = pd.concat([complete_df], ignore_index=True)
# Step 3: Apply filters through df based logic, currently 4 filters exist (volume, total, general min and general max)

# First filter for pipette volume constraints, optional Volume Restriction to select certain components for filter application ("stock" must be in column name)
complete_df_f1 = CreateSamples.pipette_volume_restriction_df(complete_df, 30, 1000, experiment_csv_dict['Volume Restriction']) # last argument is optional

# Second filter for overall total volume_restriction, call max destination well volume ("Total Sample Volume" must be in column name)
# max_dest_well_volume = ALH.find_max_dest_volume_labware(experiment_csv_dict, custom_labware_dict)
complete_df_f2 = CreateSamples.total_volume_restriction_df(complete_df_f1,800)

#Thrid filter for any general max or min filtering you would like
final_complete_df = complete_df_f2#CreateSamples.general_max_restriction(complete_df_f2, 360, 'pfh-ethanol-stock uL')
final_complete_df

,dppc wtf,dspepeg200 wtf,pfh wtf,ethanol wtf,water wtf,dppc-ethanol-stock uL,dspepeg2000-ethanol-stock uL,pfh-ethanol-stock uL,ethanol-stock uL,water-stock uL,Total Sample Volume uL
3,0.0001,0.000035,0.000,0.157895,0.841970,40.000002,33.267986,0.000000,49.914780,673.576211,796.758979
4,0.0001,0.000035,0.000,0.210526,0.789339,40.000002,33.267986,0.000000,89.655321,631.470947,794.394256
5,0.0001,0.000035,0.000,0.263158,0.736707,40.000002,33.267986,0.000000,128.618832,589.365684,791.252504
6,0.0001,0.000035,0.000,0.315789,0.684076,40.000002,33.267986,0.000000,166.731650,547.260421,787.260059
7,0.0001,0.000035,0.000,0.368421,0.631444,40.000002,33.267986,0.000000,203.920303,505.155158,782.343449
...,...,...,...,...,...,...,...,...,...,...,...
375,0.0001,0.000035,0.001,0.736842,0.262023,40.000002,33.267986,202.531646,231.028321,209.618316,716.446270
376,0.0001,0.000035,0.001,0.789474,0.209391,40.000002,33.267986,202.531646,258.807338,167.513053,702.120024
377,0.0001,0.000035,0.001,0.842105,0.156760,40.000002,33.267986,202.531646,285.312365,125.407789,686.519788
378,0.0001,0.000035,0.001,0.894737,0.104128,40.000002,33.267986,202.531646,310.572769,83.302526,669.674929


## Step 4a (Optional): Visual

In [258]:
%matplotlib qt
ographing.xy_scatter_df_compare(complete_df, final_complete_df, 'ethanol wtf', 'pfh wtf')
# ographing.xy_scatter_df(final_complete_df, 'ethanol wtf', 'pfh molf')

## Step 5: Finalize and Call Seperate Concentration and Volume Dataframes

In [259]:
final_wtf_df = CreateSamples.isolate_common_column(final_complete_df, 'wtf')
final_volume_df = CreateSamples.isolate_common_column(final_complete_df, 'stock')
final_volume_df

,dppc-ethanol-stock uL,dspepeg2000-ethanol-stock uL,pfh-ethanol-stock uL,ethanol-stock uL,water-stock uL
3,40.000002,33.267986,0.000000,49.914780,673.576211
4,40.000002,33.267986,0.000000,89.655321,631.470947
5,40.000002,33.267986,0.000000,128.618832,589.365684
6,40.000002,33.267986,0.000000,166.731650,547.260421
7,40.000002,33.267986,0.000000,203.920303,505.155158
...,...,...,...,...,...
375,40.000002,33.267986,202.531646,231.028321,209.618316
376,40.000002,33.267986,202.531646,258.807338,167.513053
377,40.000002,33.267986,202.531646,285.312365,125.407789
378,40.000002,33.267986,202.531646,310.572769,83.302526


## Step 6 (Optional): Calculate Stock Prep Information

In [260]:
chem_database_path = r"Chemical Database.xlsx"
stock_prep_df = CreateSamples.calculate_stock_prep_df(experiment_csv_dict, final_volume_df, chem_database_path, buffer_pct=290)
# pd.set_option('display.float_format', lambda x: '%.2e' % x)
stock_prep_df
# so acutally have it print with concentration, just in case you go iver 

36.97200214659444
30.749599001960426


,dppc-ethanol-stock L,dspepeg2000-ethanol-stock L,pfh-ethanol-stock L,ethanol-stock L,water-stock L
solute mass g,0.073944,0.025742,0.382640,0.000000,0.000000
solute volume L,0.000000,0.000000,0.000226,0.000000,0.000000
solvent mass g,29.182001,24.270658,76.145365,185.986405,278.513056
solvent volume L,0.036972,0.030750,0.096472,0.235635,0.278513


In [261]:
def check_for_distribute(list1, min_val, max_val): 
    return(all(max_val >= x >= min_val or x == 0 for x in list1)) 

In [262]:
list1 = [10,34,67,3,0]

check_for_distribute(list1, 3,67)

True

## Step 7: Set up Ranges for Stocks

In [263]:
protocol = simulate.get_protocol_api('2.0', extra_labware=custom_labware_dict)
max_vol = 18000 
stock_ranges = ALH.stock_well_ranges(final_volume_df, max_vol) # set up volumes orders
stock_ranges # make sure there is an assertion when number of ranges exceeds number sof well

C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults


[[[0, 237]],
 [[0, 237]],
 [[0, 201], [201, 237]],
 [[0, 61], [61, 129], [129, 207], [207, 237]],
 [[0, 49], [49, 106], [106, 169], [169, 237]]]

In [264]:
# Step 8: Simulate/Execute

In [267]:
final_volume_df

,dppc-ethanol-stock uL,dspepeg2000-ethanol-stock uL,pfh-ethanol-stock uL,ethanol-stock uL,water-stock uL
3,40.000002,33.267986,0.000000,49.914780,673.576211
4,40.000002,33.267986,0.000000,89.655321,631.470947
5,40.000002,33.267986,0.000000,128.618832,589.365684
6,40.000002,33.267986,0.000000,166.731650,547.260421
7,40.000002,33.267986,0.000000,203.920303,505.155158
...,...,...,...,...,...
375,40.000002,33.267986,202.531646,231.028321,209.618316
376,40.000002,33.267986,202.531646,258.807338,167.513053
377,40.000002,33.267986,202.531646,285.312365,125.407789
378,40.000002,33.267986,202.531646,310.572769,83.302526


In [265]:
# please for the love of a higher power add easier way to call distribute function, and modularize the way we call commands,
# it should be easy since you already have the set up of pipettes and labware seperate from volume handinling commands, jsut need ot make more neater

In [266]:
protocol = simulate.get_protocol_api('2.0', extra_labware=custom_labware_dict)
loaded_dict = ALH.loading_labware(protocol, experiment_csv_dict) # the protocol above has been modified globally!
info = ALH.pipette_stock_volumes(protocol, loaded_dict, final_volume_df, stock_ranges)

C:\Users\Edwin\.opentrons\robot_settings.json not found. Loading defaults
C:\Users\Edwin\.opentrons\deck_calibration.json not found. Loading defaults


Switching to next stock. This is Stock 0at position 1
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Distributing [40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.00000232

Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Distributing [40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.00

Switching to next stock. This is Stock 2at position 4
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Distributing [40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.00000232

Switching to next stock. This is Stock 3at position 6
Switching to next stock. This is Stock 3at position 7
Switching to next stock. This is Stock 3at position 8
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Distributing [40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.

Dispensing 97.57682758661504 uL into C7 of Falcon 48 Well Plate 1500 ÂµL on 5 at 300.0 uL/sec
Transferring 133.7698733836482 from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to C8 of Falcon 48 Well Plate 1500 ÂµL on 5
Aspirating 133.7698733836482 uL from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 150.0 uL/sec
Dispensing 133.7698733836482 uL into C8 of Falcon 48 Well Plate 1500 ÂµL on 5 at 300.0 uL/sec
Transferring 168.89973157512364 from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to D1 of Falcon 48 Well Plate 1500 ÂµL on 5
Aspirating 168.89973157512364 uL from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 150.0 uL/sec
Dispensing 168.89973157512364 uL into D1 of Falcon 48 Well Plate 1500 ÂµL on 5 at 300.0 uL/sec
Transferring 202.90451931460177 from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to D2 of Falcon 48 Well Plate 1500 ÂµL on 5
Aspirating 202.90451931460177 uL from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 150.0 uL/

Switching to next stock. This is Stock 4at position 10
Switching to next stock. This is Stock 4at position 11
Switching to next stock. This is Stock 4at position 12
Picking up tip from A1 of Opentrons 96 Tip Rack 300 µL on 10
Distributing [40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 40.000002322400135, 

Transferring 451.6161125064709 from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A7 of Falcon 48 Well Plate 1500 ÂµL on 5
Aspirating 451.6161125064709 uL from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 500.0 uL/sec
Dispensing 451.6161125064709 uL into A7 of Falcon 48 Well Plate 1500 ÂµL on 5 at 1000.0 uL/sec
Returning tip
Dropping tip into G1 of Opentrons 96 Tip Rack 1000 µL on 11
Picking up tip from D2 of Opentrons 96 Tip Rack 300 µL on 10
Transferring 32.9097040268708 from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to A8 of Falcon 48 Well Plate 1500 ÂµL on 5
Aspirating 32.9097040268708 uL from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 at 150.0 uL/sec
Dispensing 32.9097040268708 uL into A8 of Falcon 48 Well Plate 1500 ÂµL on 5 at 300.0 uL/sec
Transferring 71.02252188310338 from B2 of 20mLscintillation 12 Well Plate 18000 ÂµL on 1 to B1 of Falcon 48 Well Plate 1500 ÂµL on 5
Aspirating 71.02252188310338 uL from B2 of 20mLscintillation 12 Well

In [ ]:
## Step 9: Uploaded to Google Drive

In [ ]:
CreateSamples.create_csv(r"C:\Users\Edwin\Desktop\test", info['info concat'], final_wtf_df.values, experiment_csv_dict)
df = pd.read_csv(r"C:\Users\Edwin\Desktop\test")
